In [1]:
from utils import roi_utils
import numpy as np
import copy 

In [3]:

for ss in np.arange(1,9):

    roi_def = roi_utils.nsd_roi_def(subject=ss, verbose=True)
    n_each = roi_def.get_sizes()
    if ss==1:
        nvox = np.zeros((8, roi_def.n_rois),dtype=int)
        
    nvox[ss-1,:] = n_each
    
    print(roi_def.nii_shape)


[1, 2, 3, 4, 5, 6, 7]
['V1v', 'V1d', 'V2v', 'V2d', 'V3v', 'V3d', 'hV4']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
['V1v', 'V1d', 'V2v', 'V2d', 'V3v', 'V3d', 'hV4', 'VO1', 'VO2', 'PHC1', 'PHC2', 'TO2', 'TO1', 'LO2', 'LO1', 'V3B', 'V3A', 'IPS0', 'IPS1', 'IPS2', 'IPS3', 'IPS4', 'IPS5', 'SPL1', 'FEF']
[1, 2, 3, 4, 5]
['OFA', 'FFA-1', 'FFA-2', 'mTL-faces', 'aTL-faces']
[1, 2, 3]
['OPA', 'PPA', 'RSC']
[1, 2, 3, 4]
['EBA', 'FBA-1', 'FBA-2', 'mTL-bodies']

Volume space: ROI defs are located at: /lab_data/tarrlab/common/datasets/NSD/nsddata/ppdata/subj01/func1pt8mm/roi

3794 voxels of overlap between kastner and prf definitions, using prf defs
unique values in retino labels:
[-1.  0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16.
 17. 18. 19. 20. 21. 22. 23. 24. 25.]
unique values in face labels:
[-1.  0.  1.  2.  3.  5.]
unique values in place labels:
[-1.  0.  1.  2.  3.]
unique values in body labels:
[-1.  0.  1.  2.  3.]
198

In [3]:
roi_def.nii_shape

array([ 80, 103,  78])

In [10]:
import pandas as pd
pd.DataFrame(data=nvox, columns=roi_def.roi_names)

,V1,V2,V3,hV4,VO1-2,PHC1-2,LO1-2,TO1-2,V3ab,IPS0-5,...,FFA-2,mTL-faces,aTL-faces,OPA,PPA,RSC,EBA,FBA-1,FBA-2,mTL-bodies
0,2392,2096,1674,721,482,382,488,339,965,2155,...,310,0,159,1611,1033,566,2971,396,430,0
1,1630,1509,1574,522,424,380,506,419,844,2179,...,529,0,275,1381,994,813,3439,0,1217,0
2,2275,1732,1457,524,498,351,530,522,829,2304,...,396,0,159,1330,1269,838,3518,875,457,0
3,1526,1301,1172,610,462,351,419,369,710,2273,...,463,163,0,1235,960,813,3288,137,524,166
4,1705,1455,1350,704,591,420,345,311,706,1725,...,455,103,148,1332,1221,771,4587,460,508,319
5,1771,1739,1938,845,695,485,715,403,883,2074,...,485,0,80,1362,1229,845,4126,985,301,167
6,1926,1475,1348,478,459,305,406,391,739,1453,...,138,80,231,1083,912,694,3062,0,552,0
7,1791,1494,1445,543,489,395,442,504,654,2091,...,648,273,203,1360,961,799,3184,433,488,143


In [2]:
for ss in np.arange(1,9):

    print('\nSubject %d'%ss)
    roi_def = roi_utils.nsd_roi_def(subject=ss, verbose=False)
    roi_def.print_overlap()


Subject 1
V1: 2392 vox total
V2: 2096 vox total
V3: 1674 vox total
    32 vox overlap with OFA
    10 vox overlap with OPA
hV4: 721 vox total
    146 vox overlap with OFA
    7 vox overlap with FFA-1
    20 vox overlap with FBA-1
VO1-2: 482 vox total
    22 vox overlap with FFA-1
    77 vox overlap with PPA
    37 vox overlap with FBA-1
PHC1-2: 382 vox total
    282 vox overlap with PPA
LO1-2: 488 vox total
    9 vox overlap with OFA
    206 vox overlap with OPA
    114 vox overlap with EBA
TO1-2: 339 vox total
    3 vox overlap with OFA
    39 vox overlap with OPA
    329 vox overlap with EBA
V3ab: 965 vox total
    468 vox overlap with OPA
IPS0-5: 2155 vox total
    234 vox overlap with OPA
SPL1: 164 vox total
FEF: 72 vox total
OFA: 355 vox total
    32 vox overlap with V3
    146 vox overlap with hV4
    9 vox overlap with LO1-2
    3 vox overlap with TO1-2
    15 vox overlap with EBA
FFA-1: 484 vox total
    7 vox overlap with hV4
    22 vox overlap with VO1-2
    179 vox overlap 